# Demonstration of the Entmoot API

This notebook includes comparisons of the Entmoot strategy to other Bofire 
strategies.

## Single objective Bayesian Optimization strategy

This section includes a comparison to the Sobo strategy given in the "Getting 
Started" docs. The API is identical, with additional options given to the 
data model. Note that the EntingStrategy only supports one candidate, as 
each generated candidate is optimal (so generating multiple would generate 
duplicates).

In [1]:
from bofire.benchmarks.single import Himmelblau
import bofire.strategies.mapper as strategy_mapper

benchmark = Himmelblau()

samples = benchmark.domain.inputs.sample(10)
experiments = benchmark.f(samples, return_complete=True)

c:\users\tobyb\phd\bofire\bofire\utils\cheminformatics.py:19: UserWarning: mordred not installed. Mordred molecular descriptors cannot be used.
  warnings.warn(
c:\users\tobyb\phd\bofire\bofire\surrogates\xgb.py:12: UserWarning: xgboost not installed, BoFire's `XGBoostSurrogate` cannot be used.
  warnings.warn("xgboost not installed, BoFire's `XGBoostSurrogate` cannot be used.")


In [8]:
from bofire.data_models.strategies.api import SoboStrategy
from bofire.data_models.acquisition_functions.api import qNEI

sobo_strategy_data_model = SoboStrategy(domain=benchmark.domain, acquisition_function=qNEI())
sobo_strategy = strategy_mapper.map(sobo_strategy_data_model)

sobo_strategy.tell(experiments=experiments)
sobo_strategy.ask(candidate_count=2)


,x_1,x_2,y_pred,y_sd,y_des
0,-2.89257,1.616752,95.211661,136.329898,-95.211661


In [2]:
from bofire.data_models.strategies.api import EntingStrategy

enting_params = {"unc_params": {"dist_metric": "l1", "acq_sense": "exploration"}}
solver_params = {"solver_name": "gurobi"}

enting_strategy_data_model = EntingStrategy(domain=benchmark.domain, enting_params=enting_params, solver_params=solver_params,
                                            learn_from_candidates_coeff=-10.0)
enting_strategy = strategy_mapper.map(enting_strategy_data_model)

enting_strategy.tell(experiments=experiments)
enting_strategy.ask(candidate_count=5)

C:\Users\tobyb\phd\entmoot\entmoot\models\mean_models\tree_ensemble.py:23: UserWarning: No 'train_params' for tree ensemble training specified. Switch training to default params!
  warnings.warn(


a> []
b> []
a> []
b> []
a> []
b> []
a> []
b> []
a> []
b> []
a> []


,x_1,x_2,y_pred,y_sd,y_des
0,6.00000,0.000000,160.939420,0.464808,-160.939420
0,5.65832,-1.388620,161.098815,0.268507,-161.098815
0,5.48748,-0.523470,161.139054,0.207765,-161.139054
0,6.00000,-0.870625,158.476108,0.189261,-158.476108
0,2.33895,-0.830720,176.175154,0.274093,-176.175154


When `candidate_count > 1`, the strategy will fit to each candidate in order to produce the next candidate. This means that sequential calls to `EntingStrategy.ask()` will tend to converge as the number of generated candidates (or "phantom" datapoints) exceeds the size of the training data.

In [12]:
enting_strategy.ask(candidate_count=5)

,x_1,x_2,y_pred,y_sd,y_des
0,6.00000,4.706701,25.869349,0.062000,-25.869349
0,5.94493,4.592220,27.104200,0.056751,-27.104200
0,5.93575,4.858930,26.214987,0.071944,-26.214987
0,6.00000,4.612508,25.935602,0.056035,-25.935602
0,5.98165,4.752829,25.892338,0.051832,-25.892338


## Multi Objective BO

In [4]:
from bofire.benchmarks.multi import DTLZ2
import bofire.strategies.mapper as strategy_mapper

benchmark = DTLZ2(dim=2, num_objectives=2)

samples = benchmark.domain.inputs.sample(10)
experiments = benchmark.f(samples, return_complete=True)

In [5]:
enting_params = {"unc_params": {"dist_metric": "l1", "acq_sense": "exploration"}}
solver_params = {"solver_name": "gurobi"}

enting_strategy_data_model = EntingStrategy(domain=benchmark.domain, enting_params=enting_params, solver_params=solver_params)
enting_strategy = strategy_mapper.map(enting_strategy_data_model)

enting_strategy.tell(experiments=experiments)
enting_strategy.ask(candidate_count=1)

C:\Users\tobyb\phd\entmoot\entmoot\models\mean_models\tree_ensemble.py:23: UserWarning: No 'train_params' for tree ensemble training specified. Switch training to default params!
  warnings.warn(


,x_0,x_1,f_0_pred,f_1_pred,f_0_sd,f_1_sd,f_0_des,f_1_des
0,0.0,1.0,0.989465,0.191064,0.573843,0.573843,-0.989465,-0.191064


## Maximize single objective

In [6]:
from bofire.benchmarks.single import Ackley

benchmark = Ackley()

samples = benchmark.domain.inputs.sample(10)
experiments = benchmark.f(samples, return_complete=True)

In [7]:
enting_params = {"unc_params": {"dist_metric": "l1", "acq_sense": "exploration"}}
solver_params = {"solver_name": "gurobi"}

enting_strategy_data_model = EntingStrategy(domain=benchmark.domain, enting_params=enting_params, solver_params=solver_params)
enting_strategy = strategy_mapper.map(enting_strategy_data_model)

enting_strategy.tell(experiments=experiments)
enting_strategy.ask(candidate_count=1)

C:\Users\tobyb\phd\entmoot\entmoot\models\mean_models\tree_ensemble.py:23: UserWarning: No 'train_params' for tree ensemble training specified. Switch training to default params!
  warnings.warn(


,x_1,x_2,y_pred,y_sd,y_des
0,-8.26237,32.768,19.802305,0.422772,-19.802305


## Compare domain to problem config

BoFire defines its problems using Domains, whereas ENTMOOT uses ProblemConfigs. 
This demonstrates the conversion from Domain to ProblemConfig, using the multi 
objective categorical problem defined in the ENTMOOT benchmark.

In [8]:
from bofire.data_models.domain.api import Domain, Inputs, Outputs
from bofire.data_models.features.api import (
    CategoricalInput,
    ContinuousInput,
    DiscreteInput,
    ContinuousOutput,
)
from bofire.data_models.objectives.api import MinimizeObjective


def build_multi_obj_categorical_problem(n_obj: int = 2, no_cat=False) -> Domain:
    """
    Builds a small test example which is used in Entmoot tests.
    """

    cat_feat = (
        []
        if no_cat
        else [CategoricalInput(key="x0", categories=("blue", "orange", "gray"))]
    )
    input_features = Inputs(
        features=cat_feat
        + [
            DiscreteInput(key="x1", values=[5, 6]),
            DiscreteInput(key="x2", values=[0, 1]),  # binary
            ContinuousInput(key="x3", bounds=[5.0, 6.0]),
            ContinuousInput(key="x4", bounds=[4.6, 6.0]),
            ContinuousInput(key="x5", bounds=[5.0, 6.0]),
        ]
    )

    output_features = Outputs(
        features=[
            ContinuousOutput(
                key=f"y{i}", objective=MinimizeObjective(w=1.0, bounds=[0.0, 1.0])
            )
            for i in range(n_obj)
        ]
    )

    domain = Domain(inputs=input_features, outputs=output_features)

    return domain


In [9]:
from bofire.utils.entmoot import domain_to_problem_config
import entmoot.benchmarks as ent
from entmoot.problem_config import ProblemConfig

domain = build_multi_obj_categorical_problem()
problem_config, _ = domain_to_problem_config(domain)
problem_config_ent = ProblemConfig()
ent.build_multi_obj_categorical_problem(problem_config_ent)

print(problem_config, "\n", problem_config_ent)


PROBLEM SUMMARY
---------------
features:
x0 :: Categorical :: ('blue', 'orange', 'gray') 
x1 :: Integer :: (5, 6) 
x2 :: Binary :: (0, 1) 
x3 :: Real :: (5.0, 6.0) 
x4 :: Real :: (4.6, 6.0) 
x5 :: Real :: (5.0, 6.0) 

objectives:
y0 :: MinObjective
y1 :: MinObjective 
 
PROBLEM SUMMARY
---------------
features:
feat_0 :: Categorical :: ('blue', 'orange', 'gray') 
feat_1 :: Integer :: (5, 6) 
feat_2 :: Binary :: (0, 1) 
feat_3 :: Real :: (5.0, 6.0) 
feat_4 :: Real :: (4.6, 6.0) 
feat_5 :: Real :: (5.0, 6.0) 

objectives:
obj_0 :: MinObjective
obj_1 :: MinObjective
